In [ ]:
!pip install openai

In [ ]:
import os
from google.colab import drive
import pandas as pd
import re
import random
import numpy as np
import openai
from google.colab import userdata

In [ ]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
openai_api_key = userdata.get('CY_KEY')
openai.api_key = openai_api_key

In [ ]:
nrows = 100
train_x_path = "ece1786 project/dataset/train.source"
train_y_path = "ece1786 project/dataset/train.target"
letter_positions = [1, 2]

In [ ]:
import pandas as pd
pd.read_csv(train_x_path, sep='\t',nrows=nrows,names=['X'])

,X
0,Line at an airport
1,Kind of omelet
2,Susa was its capital
3,Suffix with cavern ... or gorge?
4,Nine: Prefix
...,...
95,Absolutely out of the question!
96,"Some monitors, for short"
97,Seat of Oklahoma's Garfield County
98,Inky


# Create Dataset

In [ ]:
class DataSet():
    def __init__(self, train_x_path, train_y_path, nrows=nrows, letter_positions=[0], random_letters=False):
        self.train_X = pd.read_csv(train_x_path, sep='\t',nrows=nrows*2,names=['X']).tail(nrows).reset_index()
        self.train_y = pd.read_csv(train_y_path, sep='\t', nrows=nrows*2, names=['y']).tail(nrows).reset_index()

        self.train_X['num_of_char'] = self.train_y['y'].apply(
            lambda train_y: len(train_y.replace(" ", ""))
        )
        self.train_X['included_letters'] = self.train_y['y'].apply(
            lambda train_y: self.include_letters(train_y.replace(" ", ""),
                                                 letter_positions=letter_positions,
                                                 random_letters=random_letters)
        )

    def include_letters(self, y, letter_positions=[0], random_letters=False):
        included_letters = {}
        if random_letters:
            counter = 0
            for x in range(len(y)):
              included = random.uniform(0, 1)
              if (included < 0.5) & (counter<(len(y)-1)):
                counter +=1
                included_letters[x]=y[x]
        else:
            for l in letter_positions:
                if l > len(y)-1:
                    break
                included_letters[l] = y[l]

        return included_letters

train_dataset = DataSet(train_x_path, train_y_path, nrows, random_letters=True)
train_X = train_dataset.train_X
train_y = train_dataset.train_y

In [ ]:
train_X

,X,num_of_char,included_letters
0,Line at an airport,5,"{0: 'l', 1: 'i'}"
1,Kind of omelet,8,"{0: 'e', 1: 'g', 4: 'h'}"
2,Susa was its capital,4,"{1: 'l', 3: 'm'}"
3,Suffix with cavern ... or gorge?,3,"{0: 'o', 2: 's'}"
4,Nine: Prefix,5,"{0: 'e', 1: 'n', 3: 'e'}"
...,...,...,...
95,Absolutely out of the question!,7,"{2: 's', 5: 'e'}"
96,"Some monitors, for short",4,{1: 'c'}
97,Seat of Oklahoma's Garfield County,4,"{1: 'n', 2: 'i', 3: 'd'}"
98,Inky,4,{2: 'o'}


In [ ]:
train_y

,y
0,limos
1,egg white
2,elam
3,ous
4,en nea
...,...
95,no siree
96,lcds
97,enid
98,ebon


# ClueGPT Logic

### Helper functions

In [ ]:
def call_gpt(x,prompt, MaxToken, outputs=1, system_msg='you are a helpful assistant', top_p=1,temp=1.0):

    response = openai.chat.completions.create(
        model="gpt-4",
        # generated output can have "max_tokens" number of tokens
        max_tokens=MaxToken,
        temperature=temp,
        top_p=top_p,
        messages=[{"role": "system", "content": system_msg},
                  {"role": "user", "content": prompt+x}],
        n = outputs
    )

    return response.choices[0].message.content
def meets_requirements(x,response):
    cleaned_response = re.sub('[\W_]+', '', response).lower()

    if len(cleaned_response) == x['num_of_char']:
      print(x['included_letters'])
      for key, val in x['included_letters'].items():
        if cleaned_response[key].lower() != x['included_letters'][key].lower():
            print(cleaned_response[key])
            print(x['included_letters'][key])
            return False
      print(cleaned_response)
      return cleaned_response
    return False

def correct_guess(y,chosen_response):
    print("selected: ", chosen_response.lower())
    print("answer: ", y['y'].replace(" ", "").lower())
    if chosen_response.lower() == y['y'].replace(" ", "").lower():
      print("correct!")
      return True
    else:
      print("incorrect..")
      return False

def prompt_update(responses):
    start = " and is not '"
    responses = list(set(responses))
    if len(responses)==1:
      add = responses[0]+"'"
    else:
      first = "', '".join(responses[:-1])
      add = "' or '".join([first,responses[-1]])+"'"
    return start + add


def get_results_step_1(x,y,prompt, clue, MaxToken, outputs, results_log):
    system_msg = "You are a helpful assistant who guess answers to crossword clues without any additional text."
    temp = 1.2
    top_p = 1
    responses = []
    results_log = pd.DataFrame(columns =['Prompt','X', 'generated_return',
                                         'max_tokens', 'temperature', 'top_p', 'selected', 'correct',
                                         'system_msg', 'answer', 'clue'])
    add_to_prompt = ""
    for i in range(outputs):
      response = call_gpt(x['X'],prompt+add_to_prompt+":",
                                MaxToken, outputs=1,
                                system_msg=system_msg,
                                top_p=top_p,temp=temp)

      # clean response first, then append
      if len(response.split(' ')) > 1:
          response_list = response.split(' ')
          response_list = [re.sub('[\W_]+', '', r).lower() for r in response_list]
          for r in response_list:
              if meets_requirements(x, r):
                  response = r
                  print("met requirements GPT response: ", response)
                  break
              else:
                  continue
      else:
          response = re.sub('[\W_]+', '', response).lower()
      responses.append(response)
      add_to_prompt =  prompt_update(responses)
      selected = meets_requirements(x,responses[i])
      if selected != False:
        break
    correct = correct_guess(y, selected) if selected else False
    new_row = {'Prompt':prompt + add_to_prompt +": "+ "<clue>",
               'X':x,
               'generated_return':responses,
               'max_tokens':MaxToken,'temperature':temp,'top_p':top_p, 'selected': selected,
               'correct':correct,
               "system_msg":system_msg,
               'answer':y['y'], 'clue': clue}
    results_log = results_log.append(new_row, ignore_index=True)
    print("results_log: ", results_log)
    return new_row,results_log


In [ ]:
def include_letter_positions(x):
  add = ""
  word_place = [x['included_letters'][key] + " as letter " + str(key+1) for key in x['included_letters']]
  if x['included_letters']!={}:
    add = " and includes letters " + ', '.join(word_place)
  return add

### 1. Guess an answer and save results in csv.

In [ ]:
# First part: getting an answer
max_token = 8
outputs = 10
def get_answers(train_X, results_file_name):
  df = pd.DataFrame()
  for n in range(train_X.shape[0]):
      print(train_X.loc[n, 'X'])
      prompt = "Guess an answer to the following crossword clue which is {0} \
      letters long {1}.".format(train_X.loc[n,'num_of_char'], include_letter_positions(train_X.loc[n,:]))
      new_row, new_df = get_results_step_1(train_X.loc[n,:],train_y.loc[n,:],prompt,train_X.loc[n, 'X'], MaxToken=max_token, outputs=outputs, results_log=df)
      df = pd.concat([df, new_df])
  df.to_csv(results_file_name)
  return df


### 2. Generate a clue based on guessed answer

In [ ]:
import os
# Second part: generating a clue based on First part
def prompt_to_gen_clue(row):
  '''
  row: a row in dataframe
  '''
  prompt_template = f"Given the word \"{row['selected']}\", \
  generate another crossword clue that is not \"{row['clue']}\". Output your clue in the format like so,\
  Another clue is: <your clue here>."

  return prompt_template

def get_gpt_clue(row, prompt_col):
  if row['correct'] is False:
    return "No clue available."
  else:
    response = openai.chat.completions.create(
      model="gpt-4",
      messages=[
        {
          "role": "user",
          "content": f"{row[prompt_col]}"
        }
      ],
      temperature=1,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    return response.choices[0].message.content

def generate_clue(dataset_df, results_filename, results_folder='ece1786 project/Final Report/'):
  dataset_df["gpt_clue"] = dataset_df.apply(get_gpt_clue, prompt_col='clue_prompt', axis=1)
  dataset_df.to_csv(os.path.join(results_folder, results_filename))
  return dataset_df


## Experiments

In [ ]:
# Supply random letters
train_dataset = DataSet(train_x_path, train_y_path, nrows=50, random_letters=True)
train_X = train_dataset.train_X.drop(['index'],axis=1)
train_y = train_dataset.train_y.drop(['index'],axis=1)

random_letters_df = get_answers(train_X, results_file_name="ece1786 project/Final Report/answers_random_50_v2.csv")


In [ ]:
random_letters_df

In [ ]:
random_letters_df["clue_prompt"] = random_letters_df.apply(prompt_to_gen_clue, axis=1)
generated_clues = generate_clue(random_letters_df, results_filename='clues_random_50_v2_attempt2.csv', results_folder='ece1786 project/Final Report/')

#Version 0 - Zero shot prompting

In [ ]:
train_dataset = DataSet(train_x_path, train_y_path, nrows=50, random_letters=True)
train_X = train_dataset.train_X.drop(['index'],axis=1)
train_y = train_dataset.train_y.drop(['index'],axis=1)

In [ ]:
import pandas as pd
def fill_prompt(row):
  '''
  row: a row in dataframe
  '''
  prompt_template = f"Givens this clue: \"{row['Clue']}\", \
  the answer has {row['num_letters']} letters, and {include_letter_positions(train_X.loc[n,:])}'. \
  Let's think step by step, when you see the clue \"{row['Clue']}\", think of answer to the clue. \
  Without supplying an answer to the clue, produce a new clue to the same answer. supply the new clue like so: \
  Another clue is: <your new clue here>."

  return prompt_template
def get_gpt_resp(row):
  response = client.chat.completions.create(
    model="gpt-4",
    messages=[
      {
        "role": "user",
        "content": f"{row['prompt']}"
      }
    ],
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response.choices[0].message.content

In [ ]:
train_X['prompt'] = train_X.apply(fill_prompt, axis=1)
train_X['version0_clue'] = train_X.apply(get_gpt_resp, axis=1)
train_X.to_csv('/content/drive/MyDrive/ece1786 project/Final Report/small_ds_gpt_baseline.csv')